In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
from IPython.display import display, Javascript

disable_js = """
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
"""

def load_ipython_extension(ip):
    display(Javascript(disable_js))
    print ("autoscrolling long output is disabled")
load_ipython_extension("")


from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2 style="text-align: center;">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

<IPython.core.display.Javascript object>

autoscrolling long output is disabled


In [3]:
repo_path = os.path.abspath(os.path.join('../..'))
display(os.listdir(repo_path))

sys.path.insert(0, repo_path)

['.git',
 '.idea',
 'bet_races_notes',
 'data',
 'f1pm',
 'ModelOne_TODOsNotes',
 'notebooks']

In [4]:
from f1pm.historicaldataprocessing.historical_data_processing_m1 import process_historical_historical_data_m1
from f1pm.historicaldataprocessing.tools import compute_historical_sub_data_set
from f1pm.probabilityestimates.pe_historical_data import ProbabilityEstimateHistoricalData
from f1pm.betstrategiesevaluations.model_strategy_zero import ModelOneStrategyZero
from f1pm.webrequests.f1com_standing_requests import request_current_drivers_standing, request_quali_results, request_startgrid_results

# Historical data processing

In [5]:
df_data, df_data_all = process_historical_historical_data_m1()
display(df_data)

,raceId,race_name,driverRef,qualifying_position,grid,position,positionOrder,driver_standing_position,constructor_standing_position,round,year,url
0,18,Australian Grand Prix - 2008,hamilton,1.0,1,1,1,1.0,1.0,1,2008.0,http://en.wikipedia.org/wiki/2008_Australian_G...
1,18,Australian Grand Prix - 2008,heidfeld,5.0,5,2,2,2.0,3.0,1,2008.0,http://en.wikipedia.org/wiki/2008_Australian_G...
2,18,Australian Grand Prix - 2008,rosberg,7.0,7,3,3,3.0,2.0,1,2008.0,http://en.wikipedia.org/wiki/2008_Australian_G...
3,18,Australian Grand Prix - 2008,alonso,12.0,11,4,4,4.0,4.0,1,2008.0,http://en.wikipedia.org/wiki/2008_Australian_G...
4,18,Australian Grand Prix - 2008,kovalainen,3.0,3,5,5,5.0,1.0,1,2008.0,http://en.wikipedia.org/wiki/2008_Australian_G...
...,...,...,...,...,...,...,...,...,...,...,...,...
25715,1089,Italian Grand Prix - 2022,kevin_magnussen,19.0,16,16,16,12.0,7.0,16,2022.0,http://en.wikipedia.org/wiki/2022_Italian_Gran...
25716,1089,Italian Grand Prix - 2022,ricciardo,8.0,4,\N,17,14.0,5.0,16,2022.0,http://en.wikipedia.org/wiki/2022_Italian_Gran...
25717,1089,Italian Grand Prix - 2022,stroll,18.0,12,\N,18,18.0,9.0,16,2022.0,http://en.wikipedia.org/wiki/2022_Italian_Gran...
25718,1089,Italian Grand Prix - 2022,alonso,10.0,6,\N,19,9.0,4.0,16,2022.0,http://en.wikipedia.org/wiki/2022_Italian_Gran...


## Loading Current driver championship standings

In [6]:
driver_standing_url = "https://www.formula1.com/en/results.html/2022/drivers.html"
current_driver_standing_table = request_current_drivers_standing(driver_standing_url)
current_driver_standing_table = current_driver_standing_table.set_index('DRIVER')
display(current_driver_standing_table)

,POS,NATIONALITY,CAR,PTS
DRIVER,,,,
Max Verstappen VER,1,NED,Red Bull Racing RBPT,391
Charles Leclerc LEC,2,MON,Ferrari,267
Sergio Perez PER,3,MEX,Red Bull Racing RBPT,265
George Russell RUS,4,GBR,Mercedes,218
Carlos Sainz SAI,5,ESP,Ferrari,202
Lewis Hamilton HAM,6,GBR,Mercedes,198
Lando Norris NOR,7,GBR,McLaren Mercedes,109
Esteban Ocon OCO,8,FRA,Alpine Renault,79
Fernando Alonso ALO,9,ESP,Alpine Renault,65


In [7]:
### 'manual' updates in current driver championship standings
# current_driver_standing_table.loc['Sergio Perez PER', 'POS'] = 3
# current_driver_standing_table.loc['Charles Leclerc LEC', 'POS'] = 2

display(current_driver_standing_table)

,POS,NATIONALITY,CAR,PTS
DRIVER,,,,
Max Verstappen VER,1,NED,Red Bull Racing RBPT,391
Charles Leclerc LEC,2,MON,Ferrari,267
Sergio Perez PER,3,MEX,Red Bull Racing RBPT,265
George Russell RUS,4,GBR,Mercedes,218
Carlos Sainz SAI,5,ESP,Ferrari,202
Lewis Hamilton HAM,6,GBR,Mercedes,198
Lando Norris NOR,7,GBR,McLaren Mercedes,109
Esteban Ocon OCO,8,FRA,Alpine Renault,79
Fernando Alonso ALO,9,ESP,Alpine Renault,65


## Loading Race grid results

In [8]:
quali_results_url = "https://www.formula1.com/en/results.html/2022/races/1134/japan/qualifying.html"
quali_results_url = "https://www.formula1.com/en/results.html/2022/races/1133/singapore/qualifying.html"
quali_results_url = "https://www.formula1.com/en/results.html/2022/races/1135/united-states/starting-grid.html"
grid_results = request_startgrid_results(quali_results_url)
grid_results = grid_results.set_index('DRIVER')
display(grid_results)

,POS,NO,CAR,TIME
DRIVER,,,,
Carlos Sainz SAI,1,55,Ferrari,1:34.356
Max Verstappen VER,2,1,Red Bull Racing RBPT,1:34.448
Lewis Hamilton HAM,3,44,Mercedes,1:34.947
George Russell RUS,4,63,Mercedes,1:34.988
Lance Stroll STR,5,18,Aston Martin Aramco Mercedes,1:35.598
Lando Norris NOR,6,4,McLaren Mercedes,1:35.690
Valtteri Bottas BOT,7,77,Alfa Romeo Ferrari,1:36.319
Alexander Albon ALB,8,23,Williams Mercedes,1:36.368
Sergio Perez PER,9,11,Red Bull Racing RBPT,1:34.645


## Processing Race day grid df

In [9]:
df_race_day_grid = ModelOneStrategyZero.compute_race_day_grid_df(current_driver_standing_table, grid_results)
display(df_race_day_grid)

,grid,driver_championship_standing
DRIVER,,
Carlos Sainz SAI,1,5
Max Verstappen VER,2,1
Lewis Hamilton HAM,3,6
George Russell RUS,4,4
Lance Stroll STR,5,15
Lando Norris NOR,6,7
Valtteri Bottas BOT,7,10
Alexander Albon ALB,8,19
Sergio Perez PER,9,3


# Strategy zero evaluation

## Grid estimates

In [10]:
from ipywidgets import Layout, Dropdown, interact, SelectMultiple, Checkbox

target_cumsum_position_dp_ge = Dropdown(options=list(range(1, 22)),
                                     description='target position or best',
                                     style={'description_width': 'initial'})

ci_dp_ge = Dropdown(options=[0.05, 0.10, 0.20, 0.32, 0.40, 0.5], description='ci', style={'description_width': 'initial'})

data_set_year_lower_threshold_dp_ge = Dropdown(options=[2000, 2004, 2008, 2012, 2016, None],
                                            description='year lower threshold',
                                           style={'description_width': 'initial'})

data_set_year_upper_threshold_dp_ge = Dropdown(options=[None, 2000, 2004, 2008, 2012, 2016, 2022],
                                            description='year upper threshold',
                                           style={'description_width': 'initial'})

data_set_round_lower_threshold_dp_ge = Dropdown(options=[5, 10, 15, None],
                                            description='round lower threshold',
                                            style={'description_width': 'initial'})

data_set_round_upper_threshold_dp_ge = Dropdown(options=[None, 10, 15],
                                            description='round upper threshold',
                                            style={'description_width': 'initial'})


# driver_championship_standing = HBox([Label('Driver Championship standing'), Dropdown(options=list(range(1, 22)))])



In [11]:
def compute_grid_estimate_fuct(target_cumsum_position, ci,
                                                    year_lower_threshold, year_upper_threshold,
                                                    round_lower_threshold, round_upper_threshold):
    df_driver_standing = current_driver_standing_table[['POS', 'CAR']]
    df_driver_standing = df_driver_standing.astype({'POS': int})
    df_driver_standing = df_driver_standing.rename(columns={'POS': 'driver_championship_standing'})
    
    # Probability estimates object ===================================================================================
    subdatset_params_dict = dict(year_lower_threshold=year_lower_threshold, year_upper_threshold=year_upper_threshold,
                                 round_lower_threshold=round_lower_threshold, round_upper_threshold=round_upper_threshold)

    print(subdatset_params_dict)
    pehd = ProbabilityEstimateHistoricalData(df_data, subdatset_params_dict)
    # ================================================================================================================
    
    # Strategy zero evaluation =======================================================================================
    mosz = ModelOneStrategyZero(pehd)
    sz_grid_prob_estimate = mosz.compute_grid_estimate(target_cumsum_position=target_cumsum_position,
                                                       current_driver_standing_table=df_driver_standing,
                                                       ci=ci,
                                                       subset_n_threshold=10)
    # ================================================================================================================
    
    # Outputs prints ---------------------------------------------
    print("=="*50)
    
    print("Inputs: \n", subdatset_params_dict, 
          "\n target_cumsum_position: ", target_cumsum_position,
         "\n ci: ", ci)
    
    print("=="*50)
    
    # Rename of columns (dor better display)
    race_target_position_prob = sz_grid_prob_estimate.race_target_position_prob.rename(columns={'driver_championship_standing': 'champ. stand.'})
    fair_bet_value = sz_grid_prob_estimate.fair_bet_value.rename(columns={'driver_championship_standing': 'champ. stand.'})
    
    display_side_by_side(race_target_position_prob, 
                         fair_bet_value, 
                         titles=['grid target position prob', 'fair bet value'])
    
    

In [12]:
_ = interact(compute_grid_estimate_fuct, 
             target_cumsum_position=target_cumsum_position_dp_ge,
             ci=ci_dp_ge,
             year_lower_threshold=data_set_year_lower_threshold_dp_ge, 
             year_upper_threshold=data_set_year_upper_threshold_dp_ge,
             round_lower_threshold=data_set_round_lower_threshold_dp_ge, 
             round_upper_threshold=data_set_round_upper_threshold_dp_ge)

interactive(children=(Dropdown(description='target position or best', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, …

### Car estimates

In [13]:
from ipywidgets import Layout, Dropdown, interact, SelectMultiple, Checkbox

ci_dp_ge_ce = Dropdown(options=[0.05, 0.10, 0.20, 0.32, 0.40, 0.5], description='ci', style={'description_width': 'initial'})

data_set_year_lower_threshold_dp_ge_ce = Dropdown(options=[2000, 2004, 2008, 2012, 2016, None],
                                            description='year lower threshold',
                                           style={'description_width': 'initial'})

data_set_year_upper_threshold_dp_ge_ce = Dropdown(options=[None, 2000, 2004, 2008, 2012, 2016, 2022],
                                            description='year upper threshold',
                                           style={'description_width': 'initial'})

data_set_round_lower_threshold_dp_ge_ce = Dropdown(options=[5, 10, 15, None],
                                            description='round lower threshold',
                                            style={'description_width': 'initial'})

data_set_round_upper_threshold_dp_ge_ce = Dropdown(options=[None, 10, 15],
                                            description='round upper threshold',
                                            style={'description_width': 'initial'})


# driver_championship_standing = HBox([Label('Driver Championship standing'), Dropdown(options=list(range(1, 22)))])



In [14]:
def compute_grid_estimate_ce_fuct(ci,
                                                    year_lower_threshold, year_upper_threshold,
                                                    round_lower_threshold, round_upper_threshold):
    df_driver_standing = current_driver_standing_table[['POS', 'CAR']]
    df_driver_standing = df_driver_standing.astype({'POS': int})
    df_driver_standing = df_driver_standing.rename(columns={'POS': 'driver_championship_standing'})

    current_driver_standing_table_ce = current_driver_standing_table.copy()
    current_driver_standing_table_ce = current_driver_standing_table.reset_index()

    # Probability estimates object ===================================================================================
    subdatset_params_dict = dict(year_lower_threshold=year_lower_threshold, year_upper_threshold=year_upper_threshold,
                                 round_lower_threshold=round_lower_threshold, round_upper_threshold=round_upper_threshold)

    print(subdatset_params_dict)
    pehd = ProbabilityEstimateHistoricalData(df_data, subdatset_params_dict)
    # ================================================================================================================

    # Strategy zero evaluation =======================================================================================
    mosz = ModelOneStrategyZero(pehd)
    sz_grid_prob_estimate = mosz.compute_grid_estimate(target_cumsum_position=1,
                                                       current_driver_standing_table=df_driver_standing,
                                                       ci=ci,
                                                       subset_n_threshold=10)
    # ================================================================================================================

    # Mapping drivers to car estimate ================================================================================
    sz_car_race_prob_estimate_cond = mosz.compute_mapping_drivers_to_car_race_estimate(
        sz_probability_estimate=sz_grid_prob_estimate,
        current_driver_standing_table=current_driver_standing_table_ce)
    # ================================================================================================================


    # Rename of columns (dor better display)
    race_target_position_prob = sz_car_race_prob_estimate_cond.race_target_position_prob
    fair_bet_value = sz_car_race_prob_estimate_cond.fair_bet_value

    display_side_by_side(race_target_position_prob, 
                         fair_bet_value, 
                         titles=['grid target position prob', 'fair bet value'])

In [15]:
_ = interact(compute_grid_estimate_ce_fuct, 
             ci=ci_dp_ge_ce,
             year_lower_threshold=data_set_year_lower_threshold_dp_ge_ce, 
             year_upper_threshold=data_set_year_upper_threshold_dp_ge_ce,
             round_lower_threshold=data_set_round_lower_threshold_dp_ge_ce, 
             round_upper_threshold=data_set_round_upper_threshold_dp_ge_ce)

interactive(children=(Dropdown(description='ci', options=(0.05, 0.1, 0.2, 0.32, 0.4, 0.5), style=DescriptionSt…

## Race Estimates

In [16]:
from ipywidgets import Layout, Dropdown, interact, SelectMultiple, Checkbox

target_cumsum_position_dp = Dropdown(options=list(range(1, 22)),
                                     description='target position or best',
                                     style={'description_width': 'initial'})

ci_dp = Dropdown(options=[0.05, 0.10, 0.20, 0.32, 0.40, 0.5], description='ci', style={'description_width': 'initial'})

data_set_year_lower_threshold_dp = Dropdown(options=[2000, 2004, 2008, 2012, 2016, None],
                                            description='year lower threshold',
                                           style={'description_width': 'initial'})

data_set_year_upper_threshold_dp = Dropdown(options=[None, 2000, 2004, 2008, 2012, 2016, 2022],
                                            description='year upper threshold',
                                           style={'description_width': 'initial'})

data_set_round_lower_threshold_dp = Dropdown(options=[5, 10, 15, None],
                                            description='round lower threshold',
                                            style={'description_width': 'initial'})

data_set_round_upper_threshold_dp = Dropdown(options=[None, 10, 15],
                                            description='round upper threshold',
                                            style={'description_width': 'initial'})


# driver_championship_standing = HBox([Label('Driver Championship standing'), Dropdown(options=list(range(1, 22)))])



In [17]:
def compute_race_estimate_fuct(target_cumsum_position, ci,
                                                    year_lower_threshold, year_upper_threshold,
                                                    round_lower_threshold, round_upper_threshold):
    df_driver_standing = current_driver_standing_table[['POS', 'CAR']]
    df_driver_standing = df_driver_standing.astype({'POS': int})
    df_driver_standing = df_driver_standing.rename(columns={'POS': 'driver_championship_standing'})
    
    # Probability estimates object ===================================================================================
    subdatset_params_dict = dict(year_lower_threshold=year_lower_threshold, year_upper_threshold=year_upper_threshold,
                                 round_lower_threshold=round_lower_threshold, round_upper_threshold=round_upper_threshold)

    print(subdatset_params_dict)
    pehd = ProbabilityEstimateHistoricalData(df_data, subdatset_params_dict)
    # ================================================================================================================
    
    # Strategy zero evaluation =======================================================================================
    mosz = ModelOneStrategyZero(pehd)
    sz_race_prob_estimate = mosz.compute_race_estimate(target_cumsum_position=target_cumsum_position,
                                                       current_driver_standing_table=df_driver_standing,
                                                       ci=ci,
                                                       subset_n_threshold=10)
    # ================================================================================================================
    
    # Outputs prints ---------------------------------------------
    print("=="*50)
    
    print("Inputs: \n", subdatset_params_dict, 
          "\n target_cumsum_position: ", target_cumsum_position,
         "\n ci: ", ci)
    
    print("=="*50)
    
    # Rename of columns (dor better display)
    race_target_position_prob = sz_race_prob_estimate.race_target_position_prob.rename(columns={'driver_championship_standing': 'champ. stand.'})
    fair_bet_value = sz_race_prob_estimate.fair_bet_value.rename(columns={'driver_championship_standing': 'champ. stand.'})
    
    display_side_by_side(race_target_position_prob, 
                         fair_bet_value, 
                         titles=['race target position prob', 'fair bet value'])
    
    

In [18]:
_ = interact(compute_race_estimate_fuct, 
             target_cumsum_position=target_cumsum_position_dp,
             ci=ci_dp,
             year_lower_threshold=data_set_year_lower_threshold_dp, 
             year_upper_threshold=data_set_year_upper_threshold_dp,
             round_lower_threshold=data_set_round_lower_threshold_dp, 
             round_upper_threshold=data_set_round_upper_threshold_dp)

interactive(children=(Dropdown(description='target position or best', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, …

### Car estimates

In [19]:
from ipywidgets import Layout, Dropdown, interact, SelectMultiple, Checkbox


ci_dp_ce = Dropdown(options=[0.05, 0.10, 0.20, 0.32, 0.40, 0.5], description='ci', style={'description_width': 'initial'})

data_set_year_lower_threshold_dp_ce = Dropdown(options=[2000, 2004, 2008, 2012, 2016, None],
                                            description='year lower threshold',
                                           style={'description_width': 'initial'})

data_set_year_upper_threshold_dp_ce = Dropdown(options=[None, 2000, 2004, 2008, 2012, 2016, 2022],
                                            description='year upper threshold',
                                           style={'description_width': 'initial'})

data_set_round_lower_threshold_dp_ce = Dropdown(options=[5, 10, 15, None],
                                            description='round lower threshold',
                                            style={'description_width': 'initial'})

data_set_round_upper_threshold_dp_ce = Dropdown(options=[None, 10, 15],
                                            description='round upper threshold',
                                            style={'description_width': 'initial'})


# driver_championship_standing = HBox([Label('Driver Championship standing'), Dropdown(options=list(range(1, 22)))])



In [20]:
def compute_race_estimate_ce_fuct(ci, year_lower_threshold, year_upper_threshold,
                                  round_lower_threshold, round_upper_threshold):
    df_driver_standing = current_driver_standing_table[['POS', 'CAR']]
    df_driver_standing = df_driver_standing.astype({'POS': int})
    df_driver_standing = df_driver_standing.rename(columns={'POS': 'driver_championship_standing'})

    current_driver_standing_table_ce = current_driver_standing_table.copy()
    current_driver_standing_table_ce = current_driver_standing_table.reset_index()

    # Probability estimates object ===================================================================================
    subdatset_params_dict = dict(year_lower_threshold=year_lower_threshold, year_upper_threshold=year_upper_threshold,
                                 round_lower_threshold=round_lower_threshold, round_upper_threshold=round_upper_threshold)

    print(subdatset_params_dict)
    pehd = ProbabilityEstimateHistoricalData(df_data, subdatset_params_dict)
    # ================================================================================================================

    # Strategy zero evaluation =======================================================================================
    mosz = ModelOneStrategyZero(pehd)
    sz_race_prob_estimate = mosz.compute_race_estimate(target_cumsum_position=1,
                                                       current_driver_standing_table=df_driver_standing,
                                                       ci=ci,
                                                       subset_n_threshold=10)
    # ================================================================================================================

    # Mapping drivers to car estimate ================================================================================
    sz_car_race_prob_estimate_cond = mosz.compute_mapping_drivers_to_car_race_estimate(
        sz_probability_estimate=sz_race_prob_estimate,
        current_driver_standing_table=current_driver_standing_table_ce)
    # ================================================================================================================


    # Rename of columns (dor better display)
    race_target_position_prob = sz_car_race_prob_estimate_cond.race_target_position_prob
    fair_bet_value = sz_car_race_prob_estimate_cond.fair_bet_value

    display_side_by_side(race_target_position_prob, 
                         fair_bet_value, 
                         titles=['race target position prob', 'fair bet value'])

In [21]:
_ = interact(compute_race_estimate_ce_fuct, 
             ci=ci_dp_ce,
             year_lower_threshold=data_set_year_lower_threshold_dp_ce, 
             year_upper_threshold=data_set_year_upper_threshold_dp_ce,
             round_lower_threshold=data_set_round_lower_threshold_dp_ce, 
             round_upper_threshold=data_set_round_upper_threshold_dp_ce)

interactive(children=(Dropdown(description='ci', options=(0.05, 0.1, 0.2, 0.32, 0.4, 0.5), style=DescriptionSt…

## Race Estimates conditioning of grid

In [22]:
from ipywidgets import Layout, Dropdown, interact, SelectMultiple, Checkbox

target_cumsum_position_dp_cog = Dropdown(options=list(range(1, 22)),
                                         description='target position or best',
                                         style={'description_width': 'initial'})

ci_dp_cog = Dropdown(options=[0.05, 0.10, 0.20, 0.32, 0.40, 0.5], description='ci', style={'description_width': 'initial'})

data_set_year_lower_threshold_dp_cog = Dropdown(options=[2000, 2004, 2008, 2012, 2016, None],
                                            description='year lower threshold',
                                           style={'description_width': 'initial'})

data_set_year_upper_threshold_dp_cog = Dropdown(options=[None, 2000, 2004, 2008, 2012, 2016, 2022],
                                            description='year upper threshold',
                                           style={'description_width': 'initial'})

data_set_round_lower_threshold_dp_cog = Dropdown(options=[5, 10, 15, None],
                                            description='round lower threshold',
                                            style={'description_width': 'initial'})

data_set_round_upper_threshold_dp_cog = Dropdown(options=[None, 10, 15],
                                            description='round upper threshold',
                                            style={'description_width': 'initial'})


# driver_championship_standing = HBox([Label('Driver Championship standing'), Dropdown(options=list(range(1, 22)))])



In [23]:
def compute_conditioning_on_grid_race_estimate_fuct(target_cumsum_position, ci,
                                                    year_lower_threshold, year_upper_threshold,
                                                    round_lower_threshold, round_upper_threshold):
    
    # Probability estimates object ===================================================================================
    subdatset_params_dict = dict(year_lower_threshold=year_lower_threshold, year_upper_threshold=year_upper_threshold,
                                 round_lower_threshold=round_lower_threshold, round_upper_threshold=round_upper_threshold)

    pehd = ProbabilityEstimateHistoricalData(df_data, subdatset_params_dict)
    # ================================================================================================================
    
    # Strategy zero evaluation =======================================================================================
    mosz = ModelOneStrategyZero(pehd)
    sz_race_prob_estimate = mosz.compute_conditioning_on_grid_race_estimate(target_cumsum_position=target_cumsum_position,
                                                                                 race_day_grid=df_race_day_grid,
                                                                                 ci=ci,
                                                                                 subset_n_threshold=1)
    # ================================================================================================================
    
    # Outputs prints ---------------------------------------------
    print("=="*50)
    
    print("Inputs: \n", subdatset_params_dict, 
          "\n target_cumsum_position: ", target_cumsum_position,
         "\n ci: ", ci)
    
    print("=="*50)
    
    # Rename of columns (dor better display)
    race_target_position_prob = sz_race_prob_estimate.race_target_position_prob.rename(columns={'driver_championship_standing': 'champ. stand.'})
    fair_bet_value = sz_race_prob_estimate.fair_bet_value.rename(columns={'driver_championship_standing': 'champ. stand.'})
    
    display_side_by_side(race_target_position_prob, 
                         fair_bet_value, 
                         titles=['race target position prob', 'fair bet value'])
    
    if target_cumsum_position == 1:
        race_target_position_prob_normalized = sz_race_prob_estimate.race_target_position_prob_normalized.rename(columns={'driver_championship_standing': 'champ. stand.'})
        fair_bet_value_normalized = sz_race_prob_estimate.fair_bet_value_normalized.rename(columns={'driver_championship_standing': 'champ. stand.'})

        display_side_by_side(race_target_position_prob_normalized, 
                         fair_bet_value_normalized, 
                         titles=['race target position prob normalized', 'fair bet value normalized'])
    
    

In [24]:
_ = interact(compute_conditioning_on_grid_race_estimate_fuct, 
             target_cumsum_position=target_cumsum_position_dp_cog,
             ci=ci_dp_cog,
             year_lower_threshold=data_set_year_lower_threshold_dp_cog, 
             year_upper_threshold=data_set_year_upper_threshold_dp_cog,
             round_lower_threshold=data_set_round_lower_threshold_dp_cog, 
             round_upper_threshold=data_set_round_upper_threshold_dp_cog)

interactive(children=(Dropdown(description='target position or best', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, …

### Car estimate

In [25]:
from ipywidgets import Layout, Dropdown, interact, SelectMultiple, Checkbox

ci_dp_cog_ce = Dropdown(options=[0.05, 0.10, 0.20, 0.32, 0.40, 0.5], description='ci', style={'description_width': 'initial'})

data_set_year_lower_threshold_dp_cog_ce = Dropdown(options=[2000, 2004, 2008, 2012, 2016, None],
                                            description='year lower threshold',
                                           style={'description_width': 'initial'})

data_set_year_upper_threshold_dp_cog_ce = Dropdown(options=[None, 2000, 2004, 2008, 2012, 2016, 2022],
                                            description='year upper threshold',
                                           style={'description_width': 'initial'})

data_set_round_lower_threshold_dp_cog_ce = Dropdown(options=[5, 10, 15, None],
                                            description='round lower threshold',
                                            style={'description_width': 'initial'})

data_set_round_upper_threshold_dp_cog_ce = Dropdown(options=[None, 10, 15],
                                            description='round upper threshold',
                                            style={'description_width': 'initial'})


# driver_championship_standing = HBox([Label('Driver Championship standing'), Dropdown(options=list(range(1, 22)))])



In [26]:
def compute_conditioning_on_grid_race_estimate_ce_fuct(ci,
                                                    year_lower_threshold, year_upper_threshold,
                                                    round_lower_threshold, round_upper_threshold):
    current_driver_standing_table_ce = current_driver_standing_table.copy()
    current_driver_standing_table_ce = current_driver_standing_table.reset_index()

    # Probability estimates object ===================================================================================
    subdatset_params_dict = dict(year_lower_threshold=year_lower_threshold, year_upper_threshold=year_upper_threshold,
                                 round_lower_threshold=round_lower_threshold, round_upper_threshold=round_upper_threshold)

    print(subdatset_params_dict)
    pehd = ProbabilityEstimateHistoricalData(df_data, subdatset_params_dict)
    # ================================================================================================================

    # Strategy zero evaluation =======================================================================================
    mosz = ModelOneStrategyZero(pehd)
    sz_race_prob_estimate = mosz.compute_conditioning_on_grid_race_estimate(target_cumsum_position=1,
                                                                                 race_day_grid=df_race_day_grid,
                                                                                 ci=ci,
                                                                                 subset_n_threshold=1)
    # ================================================================================================================

    # Mapping drivers to car estimate ================================================================================
    sz_car_race_prob_estimate_cond = mosz.compute_mapping_drivers_to_car_race_estimate(
        sz_probability_estimate=sz_race_prob_estimate,
        current_driver_standing_table=current_driver_standing_table_ce)
    # ================================================================================================================


    # Rename of columns (dor better display)
    race_target_position_prob = sz_car_race_prob_estimate_cond.race_target_position_prob
    fair_bet_value = sz_car_race_prob_estimate_cond.fair_bet_value

    display_side_by_side(race_target_position_prob, 
                         fair_bet_value, 
                         titles=['race target position prob', 'fair bet value'])

In [27]:
_ = interact(compute_conditioning_on_grid_race_estimate_ce_fuct, 
             ci=ci_dp_cog_ce,
             year_lower_threshold=data_set_year_lower_threshold_dp_cog_ce, 
             year_upper_threshold=data_set_year_upper_threshold_dp_cog_ce,
             round_lower_threshold=data_set_round_lower_threshold_dp_cog_ce, 
             round_upper_threshold=data_set_round_upper_threshold_dp_cog_ce)

interactive(children=(Dropdown(description='ci', options=(0.05, 0.1, 0.2, 0.32, 0.4, 0.5), style=DescriptionSt…

## Kelly criterium evaluation

In [28]:
p = 0.204111  # 0.197542
bo = 5.5
balance = 121 + 100
kelly_criterion_eval = ModelOneStrategyZero.eval_kelly_criterion(strategy_probability=p, bet_odds=bo)

e_balance_22_races = balance*((1 + kelly_criterion_eval.geometric_mean_rate)**22)

print("Inputs: ")
print(f" p: {p}, bo: {bo}, balance: {balance}")

print("\nKelly criterion eval: ")
display(kelly_criterion_eval._asdict())
# display(kelly_criterion_eval)

print("\nExpctation after 22 races: ")
print(" Expected value (22 races):", e_balance_22_races)
print(" Expected value (22 races) [%]:", 100*(e_balance_22_races - balance)/balance)

print("\nAmount to bet:")
print(f" To bet: {kelly_criterion_eval.f*balance}  (based on: {balance})")

Inputs: 
 p: 0.204111, bo: 5.5, balance: 221

Kelly criterion eval: 


{'f': 0.02724677777777776,
 'strategy_probability': 0.204111,
 'bet_odds_probability': 0.18181818181818182,
 'bet_odds': 5.5,
 'Strategy_odds_fair_value': 4.899294991450731,
 'geometric_mean_rate': 0.0016218161557897126,
 'is_good_bet': True}


Expctation after 22 races: 
 Expected value (22 races): 229.02101205093834
 Expected value (22 races) [%]: 3.629417217619158

Amount to bet:
 To bet: 6.021537888888885  (based on: 221)


In [29]:
p = 0.101853# 0.123273  # 0.197542
bo = 11
balance = 121 + 100
kelly_criterion_eval = ModelOneStrategyZero.eval_kelly_criterion(strategy_probability=p, bet_odds=bo)

e_balance_22_races = balance*((1 + kelly_criterion_eval.geometric_mean_rate)**22)

print("Inputs: ")
print(f" p: {p}, bo: {bo}, balance: {balance}")

print("\nKelly criterion eval: ")
display(kelly_criterion_eval._asdict())
# display(kelly_criterion_eval)

print("\nExpctation after 22 races: ")
print(" Expected value (22 races):", e_balance_22_races)
print(" Expected value (22 races) [%]:", 100*(e_balance_22_races - balance)/balance)

print("\nAmount to bet:")
print(f" To bet: {kelly_criterion_eval.f*balance}  (based on: {balance})")

Inputs: 
 p: 0.101853, bo: 11, balance: 221

Kelly criterion eval: 


{'f': 0.012038300000000002,
 'strategy_probability': 0.101853,
 'bet_odds_probability': 0.09090909090909091,
 'bet_odds': 11,
 'Strategy_odds_fair_value': 9.818071141743493,
 'geometric_mean_rate': 0.0007001655958194242,
 'is_good_bet': True}


Expctation after 22 races: 
 Expected value (22 races): 224.4293491626241
 Expected value (22 races) [%]: 1.5517417025448446

Amount to bet:
 To bet: 2.6604643000000006  (based on: 221)


In [41]:
p = 0.044043# 0.123273  # 0.197542
bo = 34
balance = 121 + 100
kelly_criterion_eval = ModelOneStrategyZero.eval_kelly_criterion(strategy_probability=p, bet_odds=bo)

e_balance_22_races = balance*((1 + kelly_criterion_eval.geometric_mean_rate)**22)

print("Inputs: ")
print(f" p: {p}, bo: {bo}, balance: {balance}")

print("\nKelly criterion eval: ")
display(kelly_criterion_eval._asdict())
# display(kelly_criterion_eval)

print("\nExpctation after 22 races: ")
print(" Expected value (22 races):", e_balance_22_races)
print(" Expected value (22 races) [%]:", 100*(e_balance_22_races - balance)/balance)

print("\nAmount to bet:")
print(f" To bet: {kelly_criterion_eval.f*balance}  (based on: {balance})")

Inputs: 
 p: 0.044043, bo: 34, balance: 221

Kelly criterion eval: 


{'f': 0.015074606060606063,
 'strategy_probability': 0.044043,
 'bet_odds_probability': 0.029411764705882353,
 'bet_odds': 34,
 'Strategy_odds_fair_value': 22.705083668233318,
 'geometric_mean_rate': 0.003268248117128003,
 'is_good_bet': True}


Expctation after 22 races: 
 Expected value (22 races): 237.447588606364
 Expected value (22 races) [%]: 7.442347785685071

Amount to bet:
 To bet: 3.33148793939394  (based on: 221)


In [36]:
bet_combinatopm = ModelOneStrategyZero.compute_uniform_bets_combination(bet_odds_list=[2.00, 15.00])

p = 0.508021 + 0.079956 # 0.123273  # 0.197542
# p = 0.5*(0.508021 + 0.079956) + 0.5*(0.568371 + 0.111111)  # 0.123273  # 0.197542

bo = bet_combinatopm.bet_odds_value
balance = 121 + 100
kelly_criterion_eval = ModelOneStrategyZero.eval_kelly_criterion(strategy_probability=p, bet_odds=bo)

e_balance_22_races = balance*((1 + kelly_criterion_eval.geometric_mean_rate)**22)

print("Inputs: ")
print(f" p: {p}, bo: {bo}, balance: {balance}")

print("\nKelly criterion eval: ")
display(kelly_criterion_eval._asdict())
# display(kelly_criterion_eval)

print("\nExpctation after 22 races: ")
print(" Expected value (22 races):", e_balance_22_races)
print(" Expected value (22 races) [%]:", 100*(e_balance_22_races - balance)/balance)

print("\nAmount to bet:")
print(f" To bet: {kelly_criterion_eval.f*balance}  (based on: {balance})")

Inputs: 
 p: 0.587977, bo: 1.7647058823529411, balance: 221

Kelly criterion eval: 


{'f': 0.0491776923076922,
 'strategy_probability': 0.587977,
 'bet_odds_probability': 0.5666666666666667,
 'bet_odds': 1.7647058823529411,
 'Strategy_odds_fair_value': 1.7007467979189663,
 'geometric_mean_rate': 0.0009290076384673718,
 'is_good_bet': True}


Expctation after 22 races: 
 Expected value (22 races): 225.56116905650993
 Expected value (22 races) [%]: 2.063877401135713

Amount to bet:
 To bet: 10.868269999999976  (based on: 221)


In [32]:
p = 0.508021# 0.123273  # 0.197542
bo = 2.0
balance = 121 + 100
kelly_criterion_eval = ModelOneStrategyZero.eval_kelly_criterion(strategy_probability=p, bet_odds=bo)

e_balance_22_races = balance*((1 + kelly_criterion_eval.geometric_mean_rate)**22)

print("Inputs: ")
print(f" p: {p}, bo: {bo}, balance: {balance}")

print("\nKelly criterion eval: ")
display(kelly_criterion_eval._asdict())
# display(kelly_criterion_eval)

print("\nExpctation after 22 races: ")
print(" Expected value (22 races):", e_balance_22_races)
print(" Expected value (22 races) [%]:", 100*(e_balance_22_races - balance)/balance)

print("\nAmount to bet:")
print(f" To bet: {kelly_criterion_eval.f*balance}  (based on: {balance})")

Inputs: 
 p: 0.508021, bo: 2.0, balance: 221

Kelly criterion eval: 


{'f': 0.01604199999999989,
 'strategy_probability': 0.508021,
 'bet_odds_probability': 0.5,
 'bet_odds': 2.0,
 'Strategy_odds_fair_value': 1.9684225652089187,
 'geometric_mean_rate': 0.00012868668089227597,
 'is_good_bet': True}


Expctation after 22 races: 
 Expected value (22 races): 221.62652078616244
 Expected value (22 races) [%]: 0.28349356839929624

Amount to bet:
 To bet: 3.5452819999999754  (based on: 221)


In [33]:
p = 0.508021# 0.123273  # 0.197542
bo = 2.1
balance = 121 + 100
kelly_criterion_eval = ModelOneStrategyZero.eval_kelly_criterion(strategy_probability=p, bet_odds=bo)

e_balance_22_races = balance*((1 + kelly_criterion_eval.geometric_mean_rate)**22)

print("Inputs: ")
print(f" p: {p}, bo: {bo}, balance: {balance}")

print("\nKelly criterion eval: ")
display(kelly_criterion_eval._asdict())
# display(kelly_criterion_eval)

print("\nExpctation after 22 races: ")
print(" Expected value (22 races):", e_balance_22_races)
print(" Expected value (22 races) [%]:", 100*(e_balance_22_races - balance)/balance)

print("\nAmount to bet:")
print(f" To bet: {kelly_criterion_eval.f*balance}  (based on: {balance})")

Inputs: 
 p: 0.508021, bo: 2.1, balance: 221

Kelly criterion eval: 


{'f': 0.06076736363636359,
 'strategy_probability': 0.508021,
 'bet_odds_probability': 0.47619047619047616,
 'bet_odds': 2.1,
 'Strategy_odds_fair_value': 1.9684225652089187,
 'geometric_mean_rate': 0.002030293882129719,
 'is_good_bet': True}


Expctation after 22 races: 
 Expected value (22 races): 231.0846018343216
 Expected value (22 races) [%]: 4.563168250824255

Amount to bet:
 To bet: 13.429587363636355  (based on: 221)


In [44]:
bet_combinatopm = ModelOneStrategyZero.compute_uniform_bets_combination(bet_odds_list=[2.10, 15.00])

p = 0.508021 + 0.079956 # 0.123273  # 0.197542
# p = 0.5*(0.508021 + 0.079956) + 0.5*(0.568371 + 0.111111)  # 0.123273  # 0.197542
p = 0.521541 + 0.086935

bo = bet_combinatopm.bet_odds_value
balance = 121 + 100
kelly_criterion_eval = ModelOneStrategyZero.eval_kelly_criterion(strategy_probability=p, bet_odds=bo)

e_balance_22_races = balance*((1 + kelly_criterion_eval.geometric_mean_rate)**22)

print("Inputs: ")
print(f" p: {p}, bo: {bo}, balance: {balance}")

print("\nKelly criterion eval: ")
display(kelly_criterion_eval._asdict())
# display(kelly_criterion_eval)

print("\nExpctation after 22 races: ")
print(" Expected value (22 races):", e_balance_22_races)
print(" Expected value (22 races) [%]:", 100*(e_balance_22_races - balance)/balance)

print("\nAmount to bet:")
print(f" To bet: {kelly_criterion_eval.f*balance}  (based on: {balance})")

Inputs: 
 p: 0.6337295, bo: 1.842105263157895, balance: 221

Kelly criterion eval: 


{'f': 0.19878328125,
 'strategy_probability': 0.6337295,
 'bet_odds_probability': 0.5428571428571428,
 'bet_odds': 1.842105263157895,
 'Strategy_odds_fair_value': 1.577960312720175,
 'geometric_mean_rate': 0.017055675831113115,
 'is_good_bet': True}


Expctation after 22 races: 
 Expected value (22 races): 320.60942495486324
 Expected value (22 races) [%]: 45.072137988625904

Amount to bet:
 To bet: 43.93110515625  (based on: 221)
